In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib qt

In [ ]:
from ICARUS.Workers.solver import Solver
from ICARUS.Workers.analysis import Analysis
from ICARUS.Software.GenuVP3.angles import runGNVPanglesParallel, runGNVPangles
from ICARUS.Software.GenuVP3.pertrubations import runGNVPpertrParallel, runGNVPpertr
from ICARUS.Software.GenuVP3.pertrubations import runGNVPsensitivity,runGNVPsensitivityParallel
from ICARUS.Software.GenuVP3.filesInterface import GNVPexe

## Define Solver

In [ ]:
gnvp3 = Solver(name = 'gnvp3',solverType = '3D', fidelity = 2)

# Define GNVP3 Analyses

In [ ]:
## Exe analysis option
options = {
    'HOMEDIR': 'Home Directory',
    "CASEDIR": 'Case Directory',
}
rerun = Analysis('gnvp3','rerun', GNVPexe, options)  
print(rerun)

In [ ]:
# Angles Serial option
options = {
    "plane":        "Plane Object",
    "Environment":  "Environment",
    "polars":       "2D Polars Name",
    "solver2D":     "2D Solver",
    "maxiter":      "Max Iterations",
    "timestep":     "Timestep",
    "Uinf":         "Velocity Magnitude",
    "angles":       "Angle to run",
}

anglesSerial = Analysis('gnvp3','Angles_Serial', runGNVPangles, options) 

In [ ]:
anglesParallel = anglesSerial << {
    'name': 'Angles_Parallel',
    'execute': runGNVPanglesParallel
    }

In [ ]:
pertrubationSerial = anglesSerial << {
    'name': 'Pertrubation_Serial',
    'execute': runGNVPpertr
    }

In [ ]:
pertrubationParallel = anglesSerial << {
    'name': 'Pertrubation_Parallel',
    'execute': runGNVPpertrParallel
    }
print(pertrubationParallel)

In [ ]:
gnvp3.addAnalyses([
    rerun,
    anglesSerial,
    anglesParallel,
    pertrubationSerial,
    pertrubationParallel
    ])

In [ ]:
analysis = gnvp3.getAvailableAnalyses()[0]
gnvp3.setAnalysis(analysis)

options = gnvp3.getOptions(analysis)

In [ ]:
from ICARUS.Database.Database_3D import Database_3D, ang2case
import os 
HOMEDIR = os.getcwd()
db = Database_3D(HOMEDIR)
plane = db.Planes["Hermes"]
CASEDIR = plane.CASEDIR + "/" + ang2case(0.) + "/"


In [ ]:
# gnvp3.run()

In [ ]:
gnvp3.getResults(analysis)

In [ ]:
options.HOMEDIR